In [3]:
%load_ext autoreload
%autoreload 2

import os
import sys

# Build an absolute path from this notebook's parent directory
module_path = os.path.abspath("../")

# Add to sys.path if not already present
if module_path not in sys.path:
    sys.path.append(module_path)

import numpy as np
from aerosandbox import Airfoil
from src import airfoil_modifications

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
airfoil = Airfoil(name="NACA0012")
airfoil.draw(backend="plotly")

In [ ]:
from src import run_xfoil
output = run_xfoil(airfoil, 0, 80, 2)
output

In [ ]:
output = airfoil.generate_polars(alpha_i=-5, alpha_f=20, alpha_step=0.25)

In [ ]:
airfoil.plot_polars()

In [ ]:
airfoil.polars[0]

In [3]:
airfoil_database_path = asb._asb_root / "geometry" / "airfoil" / "airfoil_database"

UIUC_airfoils = [
    asb.Airfoil(name=filename.stem).normalize()
    for filename in airfoil_database_path.iterdir() if filename.suffix == ".dat"
]

In [11]:
rand_airfoil = lambda: np.random.choice(UIUC_airfoils)
airfoil = rand_airfoil()

In [ ]:
airfoil.generate_polars()
airfoil.plot_polars()